In [24]:
import os
import time
import numpy as np
import mne
import pandas as pd
from pyentrp import entropy as ent
from numba import jit
import math
from entropy import *

In [1]:
def AutoCorr(data ,k = 1):
        
        data = data.values
        m = np.mean(data)
        N = len(data)
        
        res = np.sum([(data[i+k]-m) * (data[i]-m)  for i in range(N-k)]) / np.sum([(data[i+k]-m) ** (2)  for i in range(N-k)])
        return res